In [ ]:
import os
import queue
import sounddevice as sd
import vosk
import sys
import json

# Set the model path
MODEL_PATH = (
    "model"  # Change this if your model folder is different
)

# Check if model exists
if not os.path.exists(MODEL_PATH):
    print(
        "Please download the model from https://alphacephei.com/vosk/models and unpack it."
    )
    sys.exit(1)

# Load the model
model = vosk.Model(MODEL_PATH)

# Set up audio stream
samplerate = 16000  # 16kHz required
device = None  # Default microphone
q = queue.Queue()


# Callback to put audio into a queue
def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))


# Recognizer
recognizer = vosk.KaldiRecognizer(model, samplerate)

print("Real-time Speech Recognition started. Press Ctrl+C to stop.")

try:
    with sd.RawInputStream(
        samplerate=samplerate,
        blocksize=8000,
        device=device,
        dtype="int16",
        channels=1,
        callback=callback,
    ):
        while True:
            data = q.get()
            if recognizer.AcceptWaveform(data):
                result = json.loads(recognizer.Result())
                print(f"\nRecognized: {result['text']}")
            else:
                partial = json.loads(recognizer.PartialResult())
                print(f"\rPartial: {partial['partial']}", end="", flush=True)
except KeyboardInterrupt:
    print("\nStopping...")
except Exception as e:
    print(f"Error: {e}")